In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='poster')
import math
from scipy.integrate import solve_ivp

In [2]:
from numba import njit

In [3]:
import numba

levenberg optimization

In [4]:
%load_ext autoreload
%autoreload 2


In [5]:
#from model_equations_separate_NC import *
from model_equations_separate_NC_store_numba import *


In [6]:
from run_least_squares_ponly import *


In [7]:

HCOLOR = '#ffc58f'
HCOLOR_light = '#ffd2a9'
PCOLOR = '#99cc99'
PCOLOR_light = '#aad5aa'

DONCOLOR = '#7285b7'
RDONCOLOR = 'black' #'#00334D'
DINCOLOR = '#bbdaff' #'#000066'

DOCCOLOR = '#cc6666'
RDOCCOLOR = 'black' #'#00334D'
DICCOLOR = '#ff9da4' #'#000066'

SHCOLOR = '#ffa98f'
SPCOLOR = '#326232'


ccnpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR,DONCOLOR, RDONCOLOR, DINCOLOR]
ccnorder = ['Bp', 'Np', 'Bh','Nh',  'ABp', 'ABh', 'DON', 'RDON', 'DIN', ]
cccpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
cccorder = ['Bp[C]', 'Cp', 'Bh[C]', 'Ch', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
ccxpalette = [PCOLOR, HCOLOR]
ccxorder = ['Xp', 'Xh']
hnpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
hnorder = [ 'Bh', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
hcpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
hcorder = [ 'Bh[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
pnpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
pnorder = [ 'Bp', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
pcpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
pcorder = [ 'Bp[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
sns.color_palette(ccnpalette + cccpalette, )
# 


[(0.6, 0.8, 0.6),
 (0.6666666666666666, 0.8352941176470589, 0.6666666666666666),
 (1.0, 0.7725490196078432, 0.5607843137254902),
 (1.0, 0.8235294117647058, 0.6627450980392157),
 (0.19607843137254902, 0.3843137254901961, 0.19607843137254902),
 (1.0, 0.6627450980392157, 0.5607843137254902),
 (0.4470588235294118, 0.5215686274509804, 0.7176470588235294),
 (0.0, 0.0, 0.0),
 (0.7333333333333333, 0.8549019607843137, 1.0),
 (0.6, 0.8, 0.6),
 (0.6666666666666666, 0.8352941176470589, 0.6666666666666666),
 (1.0, 0.7725490196078432, 0.5607843137254902),
 (1.0, 0.8235294117647058, 0.6627450980392157),
 (0.19607843137254902, 0.3843137254901961, 0.19607843137254902),
 (1.0, 0.6627450980392157, 0.5607843137254902),
 (0.8, 0.4, 0.4),
 (0.0, 0.0, 0.0),
 (1.0, 0.615686274509804, 0.6431372549019608)]

In [8]:

 
sns.color_palette([PCOLOR, HCOLOR, PCOLOR_light, HCOLOR_light] )


[(0.6, 0.8, 0.6),
 (1.0, 0.7725490196078432, 0.5607843137254902),
 (0.6666666666666666, 0.8352941176470589, 0.6666666666666666),
 (1.0, 0.8235294117647058, 0.6627450980392157)]

In [9]:
ref_csv = 'reference_10cc_axenic.xlsx'


In [11]:

    #out_dpath = dpath
    ref_fpath =  'reference_10cc_axenic.xlsx'
    ref_pro99_fpath =  'reference_pro99_axenic.xlsx'
    json_list = []

In [12]:
    model = 'MIN'
    organism_to_tune = 'PRO'
    which_organism='ponly'
    run_id = 'tmp'

    out_dpath = 'tmp'

In [15]:
logerror=False

In [13]:
    dpath = out_dpath
    if dpath != '':
        os.makedirs(dpath, exist_ok=True)


In [14]:
    ref_df = pd.read_excel(ref_fpath)
    ref_pro99_df = pd.read_excel(ref_pro99_fpath)
    ref_df = ref_df.sort_values(['t','Sample'])
    ref_pro99_df = ref_pro99_df.sort_values(['t','Sample'])


In [16]:
    Y = pd.concat([
            ref_df['ref_Bp[N]'], 
            ref_df['ref_Bp[C]'], 
            ref_pro99_df['ref_Bp[N]'],
            ref_pro99_df['ref_Bp[C]'],
        ]).to_numpy()
    Y = np.clip(Y, a_min=4, a_max=None)
    if logerror:
        Y = np.log(Y)


In [17]:
Y

array([  4.        ,   4.        ,   4.        ,   4.        ,
         4.        ,   4.        ,   4.        ,   4.        ,
         4.        ,   4.82945553,   4.85073066,   4.72307985,
         6.51019115,   6.95696897,   7.70159868,  11.27582127,
        11.68004883,  13.55226066,  21.82828797,  20.87090692,
        22.82821929,  31.86305973,  34.19623281,  36.37338824,
        57.42158794,  61.69788997,  64.82533474,  51.37944975,
        54.0601167 ,  46.35851802,  33.97638975,  36.03298609,
        33.30976887,  20.86381521,  20.21137775,  20.26101973,
        18.23988196,  21.22549249,  19.566032  ,  18.25406538,
        18.9490531 ,  17.04847443,  18.93486968,  19.94189271,
        17.38178487,  15.42447249,  13.25440878,  14.12668929,
        14.69402621,  14.45290802,  12.63033817,  12.09136809,
        11.1056202 ,  10.08441374,   9.58090222,   8.07036768,
         7.44629707,   8.65897973,   6.88605186,   6.52437457,
         9.34687575,   5.6804609 ,   4.95001462,   6.25

In [18]:
    new_param_vals = get_param_vals_from_json_list(model, json_list)


In [19]:
    t_eval, t_end = get_t_eval_and_t_end(None, ref_df, maxday=140)
    t_eval_pro99, t_end_pro99 = get_t_eval_and_t_end(None, ref_pro99_df, maxday=140)
    (var_names, init_var_vals, intermediate_names, calc_dydt, prepare_params_tuple
        ) = get_constants_per_organism(False, which_organism)
    (_, init_var_pro99_vals, _, _, _) = get_constants_per_organism(True, which_organism)


In [21]:
    params_to_update, bounds, log_params = get_param_tuning_values(model, organism_to_tune)

    # start with the defalt params
    x0 = np.array([np.log(new_param_vals[i]) if lg else new_ [i] for i, lg in zip(params_to_update, log_params)])

    bounds_logged = [(np.log(b[0]),  np.log(b[1]))  if lg else b for b,lg in zip(bounds, log_params)]
    param_bounds =  list(zip(*bounds_logged))

    additional_params = (
        Y, params_to_update, new_param_vals, log_params, 
        calc_dydt, prepare_params_tuple, var_names, 
        init_var_vals, t_end , t_eval, ref_df, 
        init_var_pro99_vals, t_end_pro99 , t_eval_pro99, ref_pro99_df, logerror
    ) 


In [50]:
x1 = x0 + 0.1

In [51]:
    X = x1
    (Y, params_to_update, orig_param_vals, log_params, 
    calc_dydt, prepare_params_tuple, var_names, 
    init_var_vals, t_end , t_eval, ref_df, 
    init_var_pro99_vals, t_end_pro99 , t_eval_pro99, ref_pro99_df, logerror
    ) = additional_params
    print(X)

#     except BaseException as err:
#         print(f"Unexpected {err}, {type(err)}")
#         return np.zeros_like(Y)


[-13.56932805  -0.12314355  -1.84665483  -1.84665483 -10.07179686
  -8.12588671]


In [56]:
    #try:
    new_param_vals = get_params(X, params_to_update, orig_param_vals, log_params)
    par_tuple = prepare_params_tuple(new_param_vals)



In [57]:
par_tuple

(array([6.625]),
 array([10.]),
 array([4.]),
 array([[0.15776403],
        [0.1427508 ],
        [0.15776403],
        [0.1427508 ]]),
 array([[4.22546202e-05],
        [0.00000000e+00],
        [2.95782341e-04],
        [0.00000000e+00]]),
 array([3.5e-05]),
 0.0,
 array([0.01]),
 array([2.08333333e-06]),
 array([1.27913301e-06]),
 array([1.15740741e-06]),
 array([0.]),
 0.0,
 0.2502723352076287,
 array([0.]),
 array([0.88413673]),
 1.1574074074074074e-07,
 0.0,
 array([0.]))

In [55]:
par_tuple

(array([6.625]),
 array([10.]),
 array([4.]),
 array([[0.1427508],
        [0.1427508],
        [0.1427508],
        [0.1427508]]),
 array([[3.82335614e-05],
        [0.00000000e+00],
        [2.67634930e-04],
        [0.00000000e+00]]),
 array([3.5e-05]),
 0.0,
 array([0.01]),
 array([2.08333333e-06]),
 array([1.15740741e-06]),
 array([1.15740741e-06]),
 array([0.]),
 0.0,
 0.2502723352076287,
 array([0.]),
 array([0.8]),
 1.1574074074074074e-07,
 0.0,
 array([0.]))

In [39]:
    lowN_sol = run_solver_for_lsq(calc_dydt, init_var_vals, par_tuple, t_end , t_eval)
    lowN_df = solver2df_forlsq(lowN_sol, var_names, par_tuple)
    result_lowN = pd.merge_asof(ref_df, lowN_df, on='t', tolerance=1, direction='nearest')



In [43]:
    pro99_sol = run_solver_for_lsq(calc_dydt, init_var_pro99_vals, par_tuple, t_end_pro99 , t_eval_pro99)
    pro99_df = solver2df_forlsq(pro99_sol, var_names, par_tuple)
    result_pro99 = pd.merge_asof(ref_pro99_df, pro99_df, on='t', tolerance=1, direction='nearest')

In [47]:
    res = np.clip(pd.concat([
            result_lowN['Bptotal[N]'], 
            result_lowN['Bptotal[C]'], 
            result_pro99['Bptotal[N]'],
            result_pro99['Bptotal[C]'],
        ]).to_numpy(), a_min=4, a_max=None)
    if logerror:
        res = np.log(res)

In [48]:
res

array([   4.        ,    4.        ,    4.        ,    4.        ,
          4.        ,    4.        ,    5.17851508,    5.17851508,
          5.17851508,    8.11000293,    8.11000293,    8.11000293,
         17.60695597,   17.60695597,   17.60695597,   36.50182542,
         36.50182542,   36.50182542,   79.84670234,   79.84670234,
         79.84670234,   81.98953917,   81.98953917,   81.98953917,
         72.34044187,   72.34044187,   72.34044187,   65.72051389,
         65.72051389,   65.72051389,   60.65504288,   60.65504288,
         60.65504288,   54.39726573,   54.39726573,   54.39726573,
         49.79828556,   49.79828556,   49.79828556,   44.55527515,
         44.55527515,   44.55527515,   40.86209165,   40.86209165,
         40.86209165,   31.99849561,   31.99849561,   31.99849561,
         30.1769316 ,   30.1769316 ,   30.1769316 ,   27.30521687,
         27.30521687,   27.30521687,   24.74455725,   24.74455725,
         24.74455725,   22.39135619,   22.39135619,   22.39135

In [44]:
pro99_sol

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  7.680e+04 ...  7.942e+06  8.115e+06]
        y: [[ 8.929e-01  1.428e+00 ...  1.792e-01  1.468e-01]
            [ 0.000e+00  5.088e-01 ... -3.618e-32 -3.062e-32]
            ...
            [ 2.242e+03  2.235e+03 ...  2.240e+03  2.241e+03]
            [ 2.000e-01  2.000e-01 ...  2.000e-01  2.000e-01]]
      sol: None
 t_events: None
 y_events: None
     nfev: 162427
     njev: 9
      nlu: 25369

In [45]:
pro99_df

,Bp,Np,Cp,DON,RDON,DIN,DOC,RDOC,DIC,ROS,t,Bp[C],Bptotal[N],Bptotal[C]
0,0.892857,0.000000e+00,0.000000,20.000000,0.000000,8.000000e+02,132.500000,0.000000,2241.552301,0.2,0.0,5.915179,0.892857,5.915179
1,1.428018,5.087600e-01,2.656253,20.098103,0.024526,7.988334e+02,133.119956,0.154989,2234.861182,0.2,76800.0,9.460620,1.936778,12.116873
2,3.178901,1.161424e+00,6.059139,20.347275,0.086819,7.961184e+02,134.677185,0.544296,2219.390088,0.2,167100.0,21.060220,4.340325,27.119359
3,7.938676,2.901282e+00,15.135530,21.022286,0.255572,7.887750e+02,138.894738,1.598685,2178.457069,0.2,269760.0,52.593726,10.839958,67.729256
4,18.317207,6.694236e+00,34.922763,22.494079,0.623520,7.727638e+02,148.090662,3.897665,2089.979519,0.2,363540.0,121.351494,25.011443,156.274257
5,46.386519,1.695238e+01,88.437815,26.474639,1.618660,7.294607e+02,172.961646,10.115411,1851.472923,0.2,467760.0,307.310686,63.338900,395.748501
6,78.395538,2.865014e+01,149.462828,31.013932,2.753483,6.800798e+02,201.323651,17.205913,1579.774760,0.2,526620.0,519.370438,107.045680,668.833267
7,155.969415,5.699881e+01,297.347102,42.015151,5.503788,5.604057e+02,270.060406,34.390102,921.603886,0.2,603780.0,1033.297373,212.968228,1330.644476
8,303.971097,1.274002e+02,246.604086,66.802629,11.700657,3.110182e+02,419.505127,71.751282,0.031558,0.2,688500.0,2013.808515,431.371337,2260.412601
9,330.168965,1.940530e+02,82.502736,106.987820,21.746955,1.679361e+02,604.750320,118.062580,0.024404,0.2,776820.0,2187.369391,524.221982,2269.872127


In [46]:
result_pro99

,Date,day,Sample,FL,Group,full name,predicted,ref_Bp[N],ref_Bp[C],id,...,DON,RDON,DIN,DOC,RDOC,DIC,ROS,Bp[C],Bptotal[N],Bptotal[C]
0,2023-01-09 13:30:00,0.000000,Pro99_axenicA,0.335,Axenic_PRO99,Axenic_PRO99,7.982430e+06,7.127170,47.217502,Axenic_PRO99,...,20.000000,0.000000,8.000000e+02,132.500000,0.000000,2241.552301,0.2,5.915179,0.892857,5.915179
1,2023-01-09 13:30:00,0.000000,Pro99_axenicB,0.334,Axenic_PRO99,Axenic_PRO99,7.958602e+06,7.105895,47.076554,Axenic_PRO99,...,20.000000,0.000000,8.000000e+02,132.500000,0.000000,2241.552301,0.2,5.915179,0.892857,5.915179
2,2023-01-09 13:30:00,0.000000,Pro99_axenicC,0.318,Axenic_PRO99,Axenic_PRO99,7.577352e+06,6.765493,44.821390,Axenic_PRO99,...,20.000000,0.000000,8.000000e+02,132.500000,0.000000,2241.552301,0.2,5.915179,0.892857,5.915179
3,2023-01-10 10:50:00,0.888889,Pro99_axenicA,0.531,Axenic_PRO99,Axenic_PRO99,1.265275e+07,11.297096,74.843264,Axenic_PRO99,...,20.098103,0.024526,7.988334e+02,133.119956,0.154989,2234.861182,0.2,9.460620,1.936778,12.116873
4,2023-01-10 10:50:00,0.888889,Pro99_axenicB,0.460,Axenic_PRO99,Axenic_PRO99,1.096095e+07,9.786562,64.835972,Axenic_PRO99,...,20.098103,0.024526,7.988334e+02,133.119956,0.154989,2234.861182,0.2,9.460620,1.936778,12.116873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2023-04-11 11:38:00,91.922222,Pro99_axenicB,0.130,Axenic_PRO99,Axenic_PRO99,3.097660e+06,2.765767,18.323210,Axenic_PRO99,...,660.570899,160.142725,-1.852566e-31,4437.623360,1076.280840,2240.426320,0.2,1.187423,0.179234,1.699899
170,2023-04-11 11:38:00,91.922222,Pro99_axenicC,0.100,Axenic_PRO99,Axenic_PRO99,2.382815e+06,2.127513,14.094777,Axenic_PRO99,...,660.570899,160.142725,-1.852566e-31,4437.623360,1076.280840,2240.426320,0.2,1.187423,0.179234,1.699899
171,2023-04-13 11:32:00,93.918056,Pro99_axenicA,0.095,Axenic_PRO99,Axenic_PRO99,2.263674e+06,2.021138,13.390038,Axenic_PRO99,...,660.596842,160.149210,-1.835623e-31,4437.869407,1076.342352,2240.639532,0.2,0.972585,0.146805,1.392339
172,2023-04-13 11:32:00,93.918056,Pro99_axenicB,0.123,Axenic_PRO99,Axenic_PRO99,2.930863e+06,2.616842,17.336575,Axenic_PRO99,...,660.596842,160.149210,-1.835623e-31,4437.869407,1076.342352,2240.639532,0.2,0.972585,0.146805,1.392339


In [24]:
run_model(x0, additional_params)

array([   4.        ,    4.        ,    4.        ,    4.        ,
          4.        ,    4.        ,    5.17851508,    5.17851508,
          5.17851508,    8.11000293,    8.11000293,    8.11000293,
         17.60695597,   17.60695597,   17.60695597,   36.50182542,
         36.50182542,   36.50182542,   79.84670234,   79.84670234,
         79.84670234,   81.98953917,   81.98953917,   81.98953917,
         72.34044187,   72.34044187,   72.34044187,   65.72051389,
         65.72051389,   65.72051389,   60.65504288,   60.65504288,
         60.65504288,   54.39726573,   54.39726573,   54.39726573,
         49.79828556,   49.79828556,   49.79828556,   44.55527515,
         44.55527515,   44.55527515,   40.86209165,   40.86209165,
         40.86209165,   31.99849561,   31.99849561,   31.99849561,
         30.1769316 ,   30.1769316 ,   30.1769316 ,   27.30521687,
         27.30521687,   27.30521687,   24.74455725,   24.74455725,
         24.74455725,   22.39135619,   22.39135619,   22.39135

In [27]:
tmp = fun(x0, additional_params)

In [30]:
tmp = jac(x0, additional_params)

In [31]:
tmp

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [35]:
tmp[1]

array([0., 0., 0., 0., 0., 0.])

In [36]:
(tmp!=0).argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
refdf = pd.read_excel('reference_10cc_axenic.xlsx')


In [9]:
refdf.columns

Index(['day', 'Sample', 'id', 'replicate', 'transfer', 'transfer day',
       'heterotroph', 'full name', 'FL', 'Group', 'predicted', 'ref_Bp[N]',
       'ref_Bp[C]', 't'],
      dtype='object')

In [10]:
    model='MIN'
    json =[]
    dpath = 'tmp'
    pro99_mode = False
    which_organism = 'ponly'
    t_eval = None
    maxday = 140
    if dpath != '':
        os.makedirs(dpath, exist_ok=True)

    ref_csv= 'reference_10cc_axenic.xlsx'
    refdf = pd.read_excel(ref_csv)
    refdf['ref_Bp[N]'] = refdf['ref_Bp[N]'].clip(lower=4)
    refdf['ref_Bp[C]'] = refdf['ref_Bp[C]'].clip(lower=4)
    refdf['log_ref_Bp[N]'] = np.log(refdf['ref_Bp[N]'])
    refdf['log_ref_Bp[C]'] = np.log(refdf['ref_Bp[C]'])


    new_param_vals = get_param_vals_from_json_list(model, json)
    suffix = get_runid_unique_suffix(pro99_mode, which_organism, model, new_param_vals)
    t_eval, t_end = get_t_eval_and_t_end(t_eval, refdf, maxday)
    (var_names, init_var_vals, intermediate_names, calc_dydt, prepare_params_tuple
        ) = get_constants_per_organism(pro99_mode, which_organism)


In [12]:
    perr, new_param_vals, df, mse_df,  =  run_solver_from_new_params(
        new_param_vals, refdf, init_var_vals, 
        calc_dydt, prepare_params_tuple, t_end , t_eval, var_names, intermediate_names, return_dfs=True
    )


In [13]:
mse_df

,Sample,full name,Group,RMSE_Bptotal[N],RMSE_Bptotal[C],RMSE_log_Bptotal[N],RMSE_log_Bptotal[C],compare_points,RMSE,lRMSE
0,1A0,Axenic,Axenic,6.838047,67.177407,2.363544,1.855270,74.0,459.362275,4.385014
1,1B0,Axenic,Axenic,6.567046,63.688658,2.382849,1.876870,74.0,418.246340,4.472300
2,1C0,Axenic,Axenic,6.283163,64.283686,2.387140,1.876704,74.0,403.904897,4.479956


In [35]:
    par_tuple = prepare_params_tuple(new_param_vals)
    sol = run_solver(calc_dydt, init_var_vals, par_tuple, t_end , t_eval)
    df = solver2df(
        sol, var_names, par_tuple=par_tuple, 
        intermediate_names=intermediate_names, calc_dydt=calc_dydt)


In [18]:
df

,Bp,Np,Cp,DON,RDON,DIN,DOC,RDOC,DIC,ROS,...,uptakeCp,QCp,biosynthesisNp,respirationCp,biomass_breakdownCp,overflowNp,overflowCp,Bp[C],Bptotal[N],Bptotal[C]
0,8.928571e-01,0.000000e+00,0.000000e+00,20.000000,0.000000,1.000000e+02,132.500000,0.000000,2241.552301,0.2,...,8.064369e-05,6.625000,4.260421e-10,1.232330e-05,0.0,0.0,0.0,5.915179e+00,8.928571e-01,5.915179e+00
1,1.501619e+00,5.542196e-01,2.894456e+00,20.242800,0.030928,9.856329e+01,134.068550,0.194899,2233.286796,0.2,...,1.508206e-04,6.246931,1.529224e-05,2.173855e-05,0.0,0.0,0.0,9.948223e+00,2.055838e+00,1.284268e+01
2,3.161910e+00,1.180893e+00,6.163789e+00,20.748967,0.097160,9.570393e+01,137.320844,0.608422,2217.106225,0.2,...,3.179238e-04,6.242844,3.256506e-05,4.579832e-05,0.0,0.0,0.0,2.094765e+01,4.342803e+00,2.711144e+01
3,4.726309e+00,1.765308e+00,9.214309e+00,21.224876,0.159449,9.301691e+01,140.378525,0.997282,2202.170942,0.2,...,4.752217e-04,6.242837,4.868182e-05,6.845801e-05,0.0,0.0,0.0,3.131180e+01,6.491617e+00,4.052611e+01
4,9.467501e+00,3.535778e+00,1.845718e+01,22.667166,0.348222,8.487419e+01,149.645186,2.175772,2157.347960,0.2,...,9.518993e-04,6.242992,9.751454e-05,1.371314e-04,0.0,0.0,0.0,6.272220e+01,1.300328e+01,8.117937e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,6.800456e-08,5.250612e-22,2.032323e-07,110.417835,10.475022,1.662638e-15,815.276639,90.336642,2241.542808,0.2,...,7.033812e-13,9.613510,1.837774e-26,9.386035e-13,0.0,0.0,0.0,4.505302e-07,6.800456e-08,6.537625e-07
70,2.553048e-08,1.971195e-22,7.629810e-08,110.417835,10.475022,1.662633e-15,815.276639,90.336642,2241.546632,0.2,...,2.640655e-13,9.613510,6.899405e-27,3.523734e-13,0.0,0.0,0.0,1.691394e-07,2.553048e-08,2.454375e-07
71,1.417799e-08,1.094674e-22,4.237106e-08,110.417835,10.475022,1.662632e-15,815.276639,90.336642,2241.548140,0.2,...,1.466450e-13,9.613510,3.831483e-27,1.956855e-13,0.0,0.0,0.0,9.392916e-08,1.417799e-08,1.363002e-07
72,7.801750e-09,6.023681e-23,2.331561e-08,110.417835,10.475022,1.662631e-15,815.276639,90.336642,2241.549263,0.2,...,8.069465e-14,9.613510,2.108357e-27,1.076803e-13,0.0,0.0,0.0,5.168659e-08,7.801750e-09,7.500220e-08


In [19]:
    mse_df = None
    perr = 1e6 # very large number for error


In [29]:
            mse_df = compute_mse(df, refdf, refcol= ['ref_Bp[N]', 'ref_Bp[C]'], col=['Bptotal[N]', 'Bptotal[C]'], timecol='t', tolerance=100)


In [23]:
refcol= ['ref_Bp[N]', 'ref_Bp[C]']
col=['Bptotal[N]', 'Bptotal[C]']
timecol='t'
tolerance=100
    #print (x)
#newcols = [timecol] + col
df1 = df[[timecol] + col].dropna().copy()
df1[col] = df1[col].clip(lower=4)
# mse_df = refdf.groupby(['Sample', 'full name', 'Group',]
#                     ).apply(lambda y : _mse(df1, y, refcol= refcol, col=col, timecol=timecol, tolerance=tolerance))
# mse_df = mse_df.reset_index()
# return mse_df


In [26]:
def _mse(x, refdf, refcol, col, timecol, tolerance):
    #print(x.columns)
    tdf = pd.merge_asof(x, refdf[[timecol] + refcol], on=timecol, direction='nearest', tolerance=tolerance).dropna()

    mse_dict = {f'MSE_{c}': mean_squared_error(tdf[c], tdf[rc]) for c,rc in zip(col, refcol)}
    mse_dict['compare_points'] = tdf.shape[0]
    return pd.Series(mse_dict)


In [27]:
mse_df = refdf.groupby(['Sample', 'full name', 'Group',]
                    ).apply(lambda y : _mse(df1, y, refcol= refcol, col=col, timecol=timecol, tolerance=tolerance))


In [36]:
mse_df

,Sample,full name,Group,RMSE_Bptotal[N],RMSE_Bptotal[C],RMSE,compare_points
0,1A0,Axenic,Axenic,6.838047,67.177407,459.362275,74.0
1,1B0,Axenic,Axenic,6.567046,63.688658,418.246340,74.0
2,1C0,Axenic,Axenic,6.283163,64.283686,403.904897,74.0


In [37]:
        param_sensitivity =2
        organism_to_tune = 'PRO'
        number_of_runs = 5
        parameter, sensitivity_values = get_param_sensitivity_values(
            param_sensitivity, model, organism_to_tune, number_of_runs)


In [38]:
sensitivity_values

array([2.31481481e-07, 5.17608328e-07, 1.15740741e-06, 2.58804164e-06,
       5.78703704e-06])

In [60]:
        for i,v in enumerate(sensitivity_values):
            new_param_vals[parameter] = v
            run_id='tmp'
            run_id = f"{run_id}_{parameter}_{i}_{v}{suffix}"
            print(run_id)
            
            MSE_err = run_solver_from_new_params_and_save(
                new_param_vals, refdf, dpath, 
                run_id, init_var_vals, 
                calc_dydt, prepare_params_tuple, t_end , t_eval, var_names, intermediate_names,
            )
            print ('MSE:', MSE_err)


tmp_E_leakp_0_2.314814814814816e-07_lowN_ponly_MIN_h2914781599285612379
MSE: 1238.01533931916
tmp_E_leakp_1_5.176083281249514e-07_lowN_ponly_MIN_h2914781599285612379
MSE: 836.4312881915904
tmp_E_leakp_2_1.1574074074074072e-06_lowN_ponly_MIN_h2914781599285612379
MSE: 403.87895900940646
tmp_E_leakp_3_2.588041640624755e-06_lowN_ponly_MIN_h2914781599285612379
MSE: 82.40680882717648
tmp_E_leakp_4_5.787037037037031e-06_lowN_ponly_MIN_h2914781599285612379
MSE: 337.85576232800435


In [61]:
from run_shgo_ponly import *

In [62]:
os.listdir()

['.ipynb_checkpoints',
 'calc_csat.py',
 'model_equations_separate_NC_store.py',
 'model_equations_separate_NC_store_numba.py',
 'Model_Parameters Store model.xlsx',
 'model_param_sensitivity_ponly.ipynb',
 'model_store_cc debug.ipynb',
 'model_store_cc.ipynb',
 'model_store_ponly.ipynb',
 'model_store_ponly_find_lsq_x0.ipynb',
 'reference_10cc.xlsx',
 'reference_10cc_axenic.xlsx',
 'reference_pro99_axenic.xlsx',
 'results',
 'results_postprocess.py',
 'run_files',
 'run_least_squares_het.py',
 'run_least_squares_ponly.py',
 'run_monte.py',
 'run_shgo_ponly.py',
 'tmp',
 'VPRO',
 '__pycache__']

In [63]:

    out_dpath = dpath
    ref_fpath =  'reference_10cc_axenic.xlsx'
    ref_pro99_fpath =  'reference_pro99_axenic.xlsx'
    json_list = []

In [18]:
    ref_df = pd.read_excel(ref_fpath)
    ref_pro99_df = pd.read_excel(ref_pro99_fpath)
    ref_df = ref_df.sort_values(['t','Sample'])
    ref_pro99_df = ref_pro99_df.sort_values(['t','Sample'])

    Y = pd.concat([ref_df['ref_Bp'], ref_pro99_df['ref_Bp']]).to_numpy()
    Y = np.clip(Y, a_min=4, a_max=None)

    new_param_vals = get_param_vals_from_json_list(model, json_list)
    #TODO
    t_eval, t_end = get_t_eval_and_t_end(None, refdf, maxday=140)
    t_eval_pro99, t_end_pro99 = get_t_eval_and_t_end(None, ref_pro99_df, maxday=140)
    (var_names, init_var_vals, intermediate_names, calc_dydt, prepare_params_tuple
        ) = get_constants_per_organism(False, which_organism)
    (_, init_var_pro99_vals, _, _, _) = get_constants_per_organism(True, which_organism)


    params_to_update, bounds, log_params = get_param_tuning_values(model, organism_to_tune)

    # start with the defalt params
    x0 = np.array([np.log(param_vals[i]) if lg else param_vals[i] for i, lg in zip(params_to_update, log_params)])

    bounds_logged = [(np.log(b[0]),  np.log(b[1]))  if lg else b for b,lg in zip(bounds, log_params)]
    param_bounds =  bounds_logged

    # print('shgo(',
        # fun, param_bounds, 
        # #minimizer_kwargs=dict(
            # #jac=jac_for_minimize, 
    # #        hess=hess_for_minimize, 
    # #        options=dict(disp=True, maxiter=3), method='L-BFGS-B'),
    # #    options=dict(jac=
                # jac_for_minimize)

    additional_params = (
        Y, params_to_update, new_param_vals, log_params, 
        calc_dydt, prepare_params_tuple, var_names, 
        init_var_vals, t_end , t_eval, ref_df, 
        init_var_pro99_vals, t_end_pro99 , t_eval_pro99, ref_pro99_df
    ) 


In [19]:
fun(x0, additional_params)

[-13.66932805   0.8        -13.66932805  -1.94665483  -1.94665483
 -10.17179686  -8.22588671]


In [20]:
j =jac(x0, additional_params)

[-13.66932805   0.8        -13.66932805  -1.94665483  -1.94665483
 -10.17179686  -8.22588671]
[-13.66932804   0.8        -13.66932805  -1.94665483  -1.94665483
 -10.17179686  -8.22588671]
[-13.66932805   0.80000001 -13.66932805  -1.94665483  -1.94665483
 -10.17179686  -8.22588671]
[-13.66932805   0.8        -13.66932804  -1.94665483  -1.94665483
 -10.17179686  -8.22588671]
[-13.66932805   0.8        -13.66932805  -1.94665482  -1.94665483
 -10.17179686  -8.22588671]
[-13.66932805   0.8        -13.66932805  -1.94665483  -1.94665482
 -10.17179686  -8.22588671]
[-13.66932805   0.8        -13.66932805  -1.94665483  -1.94665483
 -10.17179685  -8.22588671]
[-13.66932805   0.8        -13.66932805  -1.94665483  -1.94665483
 -10.17179686  -8.2258867 ]


In [21]:
np.max(j)

In [44]:
j

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
%timeit -n 2 -r 2 fun(x0, additional_params)

4.31 s ± 39.7 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [52]:
params_to_update

['Mp', 'gammaDp', 'E_leakp', 'KINp', 'KICp', 'VmaxINp', 'VmaxICp']

In [23]:
l_bounds, u_bounds = tuple(zip(*param_bounds))

In [24]:
param_bounds

In [25]:
from scipy.stats import qmc
sampler = qmc.Sobol(d=len(params_to_update))
sample = sampler.random_base2(m=3)
sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

In [26]:
sample_scaled

array([[-14.82156512,   0.71526835, -13.10694417,  -2.82211221,
         -1.70220346, -10.28632915,  -6.86842204],
       [-13.01476924,   0.46154313, -15.10488091,  -1.05166423,
         -2.41703912,  -9.27743825,  -8.3904943 ],
       [-12.0600254 ,   0.62382078, -12.26575605,  -2.23549826,
         -2.85067081,  -9.37128692,  -8.00953066],
       [-13.8744649 ,   0.15249262, -14.28785805,  -1.68943261,
         -0.51342915, -11.59973184,  -9.73268443],
       [-14.16981765,   0.51543789, -14.68333604,  -1.400565  ,
         -1.100827  , -10.92865318,  -7.45360532],
       [-12.75929103,   0.26137694, -13.49070605,  -2.47335183,
         -3.42235012,  -8.73783444,  -9.3768345 ],
       [-13.62435109,   0.82359643, -13.91763943,  -0.43511787,
         -2.24608308, -10.03976843,  -7.32476676],
       [-15.03034071,   0.35271382, -12.6988797 ,  -3.48985519,
         -1.5218157 , -11.03598444,  -8.64435839]])

In [35]:
    SSE_list = [fun(X, additional_params) for X in sample_scaled]


[-14.82156512   0.71526835 -13.10694417  -2.82211221  -1.70220346
 -10.28632915  -6.86842204]
[-13.01476924   0.46154313 -15.10488091  -1.05166423  -2.41703912
  -9.27743825  -8.3904943 ]
[-12.0600254    0.62382078 -12.26575605  -2.23549826  -2.85067081
  -9.37128692  -8.00953066]
[-13.8744649    0.15249262 -14.28785805  -1.68943261  -0.51342915
 -11.59973184  -9.73268443]
[-14.16981765   0.51543789 -14.68333604  -1.400565    -1.100827
 -10.92865318  -7.45360532]
[-12.75929103   0.26137694 -13.49070605  -2.47335183  -3.42235012
  -8.73783444  -9.3768345 ]
[-13.62435109   0.82359643 -13.91763943  -0.43511787  -2.24608308
 -10.03976843  -7.32476676]
[-15.03034071   0.35271382 -12.6988797   -3.48985519  -1.5218157
 -11.03598444  -8.64435839]


In [41]:
sample_unlogged =np.copy(sample_scaled)
sample_unlogged[:, log_params] = np.exp(sample_unlogged[:, log_params])
sample_unlogged

array([[3.65658823e-07, 7.15268354e-01, 2.03107738e-06, 5.94801753e-02,
        1.82281431e-01, 3.40960455e-05, 1.04011704e-03],
       [2.22719137e-06, 4.61543131e-01, 2.75444160e-07, 3.49355856e-01,
        8.91852942e-02, 9.35103674e-05, 2.27015038e-04],
       [5.78625433e-06, 6.23820782e-01, 4.71031368e-06, 1.06938833e-01,
        5.78055315e-02, 8.51337572e-05, 3.32280637e-04],
       [9.42749814e-07, 1.52492625e-01, 6.23536801e-07, 1.84624249e-01,
        5.98439914e-01, 9.16854605e-06, 5.93128591e-05],
       [7.01659457e-07, 5.15437889e-01, 4.19863558e-07, 2.46457675e-01,
        3.32595913e-01, 1.79368521e-05, 5.79349102e-04],
       [2.87547976e-06, 2.61376941e-01, 1.38376010e-06, 8.43018192e-02,
        3.26356470e-02, 1.60400870e-04, 8.46627789e-05],
       [1.21065250e-06, 8.23596430e-01, 9.02913183e-07, 6.47188379e-01,
        1.05812875e-01, 4.36298751e-05, 6.59013353e-04],
       [2.96760415e-07, 3.52713823e-01, 3.05454565e-06, 3.05052892e-02,
        2.18315133e-01, 1

In [42]:
df = pd.DataFrame(sample_unlogged, columns=params_to_update)

In [43]:
df['SSE'] = SSE_list

In [44]:
df

,Mp,gammaDp,E_leakp,KINp,KICp,VmaxINp,VmaxICp,SSE
0,3.656588e-07,0.715268,2.031077e-06,0.059480,0.182281,0.000034,0.001040,1.272749e+06
1,2.227191e-06,0.461543,2.754442e-07,0.349356,0.089185,0.000094,0.000227,1.238919e+06
2,5.786254e-06,0.623821,4.710314e-06,0.106939,0.057806,0.000085,0.000332,5.678182e+05
3,9.427498e-07,0.152493,6.235368e-07,0.184624,0.598440,0.000009,0.000059,1.769270e+06
4,7.016595e-07,0.515438,4.198636e-07,0.246458,0.332596,0.000018,0.000579,4.729382e+06
5,2.875480e-06,0.261377,1.383760e-06,0.084302,0.032636,0.000160,0.000085,1.992642e+06
6,1.210653e-06,0.823596,9.029132e-07,0.647188,0.105813,0.000044,0.000659,1.817238e+06
7,2.967604e-07,0.352714,3.054546e-06,0.030505,0.218315,0.000016,0.000176,1.829472e+06


In [45]:
df.nsmallest(n=3, columns='SSE')

,Mp,gammaDp,E_leakp,KINp,KICp,VmaxINp,VmaxICp,SSE
2,5.786254e-06,0.623821,4.710314e-06,0.106939,0.057806,0.000085,0.000332,5.678182e+05
1,2.227191e-06,0.461543,2.754442e-07,0.349356,0.089185,0.000094,0.000227,1.238919e+06
0,3.656588e-07,0.715268,2.031077e-06,0.059480,0.182281,0.000034,0.001040,1.272749e+06


In [50]:
import scipy
scipy.__version__

'1.7.3'

In [65]:
sobol = 2

In [69]:
        params_to_update, bounds, log_params = get_param_tuning_values(model, organism_to_tune)
        bounds_logged = [(np.log(b[0]),  np.log(b[1]))  if lg else b for b,lg in zip(bounds, log_params)]
        param_bounds =  bounds_logged
        sample = get_sobol_sample(param_bounds, m=sobol)
        for i,X in enumerate(sample):
            sen_run_id = f"{run_id}_{i}{suffix}"
            print(sen_run_id)
            
            MSE_err = run_solver_from_X_and_save(
                X, params_to_update, new_param_vals, refdf, out_dpath, sen_run_id, 
                log_params, init_var_vals, 
                calc_dydt, prepare_params_tuple, t_end , t_eval, var_names, intermediate_names)
            print ('MSE:', MSE_err)


tmp_E_leakp_4_5.787037037037031e-06_lowN_ponly_MIN_h2914781599285612379_0_lowN_ponly_MIN_h2914781599285612379
MSE: 623.9201492409909
tmp_E_leakp_4_5.787037037037031e-06_lowN_ponly_MIN_h2914781599285612379_1_lowN_ponly_MIN_h2914781599285612379
MSE: 7686.435150257532
tmp_E_leakp_4_5.787037037037031e-06_lowN_ponly_MIN_h2914781599285612379_2_lowN_ponly_MIN_h2914781599285612379
MSE: 818.0208627730144
tmp_E_leakp_4_5.787037037037031e-06_lowN_ponly_MIN_h2914781599285612379_3_lowN_ponly_MIN_h2914781599285612379
MSE: 1469.529086598164


C:\Users\Osnat\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
